# CPDCTL Samples for Notebooks and Environments in Projects

CPDCTL is a command-line interface (CLI) you can use to manage the lifecycle of notebooks. By using the notebook CLI, you can automate the flow for creating notebooks and running notebook jobs, moving notebooks between projects in Watson Studio, and adding custom libraries to notebook runtime environments.   

This notebook begins by showing you how to install and configure CPDCTL and is then split up into four sections with examples of how to use the commands for:

- Creating notebooks and running notebook jobs
- Creating Python scripts and running script jobs
- Downloading notebooks from one project and uploading them to another project
- Adding custom libraries to a notebook runtime environment

## Table of Contents

[1. Installing and configuring CPDCTL](#part1)
- [1.1 Installing the latest version of CPDCTL](#part1.1)
- [1.2 Adding CPD cluster configuration settings](#part1.2)

[2. Demo 1: Creating a notebook asset and running a job](#part2)
- [2.1 Creating a notebook asset](#part2.1)
- [2.2 Running a job](#part2.2)

[3. Demo 2: Creating a Python script asset and running a job](#part3)
- [3.1 Creating a Python script asset](#part3.1)
- [3.2 Running a job](#part3.2)

[4. Demo 3: Downloading a notebook and uploading it to another project](#part4)
- [4.1 Downloading a notebook](#part4.1)
- [4.2 Uploading the notebook to another project](#part4.2)

[5. Demo 4: Adding additional packages to custom environment](#part5)
- [5.1 Creating a custom software specification](#part5.1)
- [5.2 Adding additional packages](#part5.2)
- [5.3 Creating a custom environment](#part5.3)

## Before you begin
Import the following libraries:

In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

/var/folders/8m/4n2vyk5x2431h86n7f402gcr0000gn/T/ipykernel_79895/1628720105.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


##  1. Installing and configuring CPDCTL <a class="anchor" id="part1"></a>

### 1.1 Installing the latest version of CPDCTL <a class="anchor" id="part1.1"></a>

To use the notebook and environment CLI commands, you need to install CPDCTL. Download the binary from the [CPDCTL GitHub respository](https://github.com/IBM/cpdctl/releases).

Download the binary and then display the version number:

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.4.68 (20231030150348)


### 1.2  Adding CPD cluster configuration settings <a class="anchor" id="part1.2"></a>

Before you can use CPDCTL, you need to add configuration settings. You only need to configure these settings once for the same IBM Cloud Pak for Data (CPD) user and cluster. Begin by entering your CPD credentials and the URL to the CPD cluster.<br>**Note**: when running this notebook inside IBM Cloud Pak for Data (CP4D) cluster, cpdctl takes advantage of [zero-configuration mode](https://github.com/IBM/cpdctl#zero-configuration) which means it can connect to the CP4D without explicit configuration. In that case the cells below that set credential and URL variables as well as cells that run `cpdctl config ...` commands can be skipped.

In [5]:
CPD_USER_NAME = #'YOUR CPD user name'
CPD_USER_PASSWORD = #'YOUR CPD user password'
CPD_URL = #'YOUR CPD CLUSTER URL'

Add "cpd_user" user to the cpdctl configuration:

In [6]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd" cluster to the cpdctl configuration:

In [7]:
! cpdctl config profile set cpd --url {CPD_URL} --user cpd_user

List available profiles:

In [8]:
! cpdctl config profile list

Name   Type      User       URL                                            IBM Cloud CLI config   Region   Current
cpd    private   cpd_user   https://cpd-dev.apps.midgard.cp.fyre.ibm.com                                   *


Switch to the profile you just created if it is not marked in the `Current` column:

In [9]:
! cpdctl config profile use cpd

Switched to profile "cpd".


List available projects in profile:

In [10]:
! cpdctl project list

...
ID                                     Name    Created                    Description
be7e848d-1a02-4282-b60a-1a728cd9cb70   test    2023-11-22T15:02:56.339Z   
277c4bc5-2665-49fc-b33d-ce17fe9610fd   test2   2023-12-05T18:07:09.694Z   


Choose the project in which you want to work:

In [11]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[0]" --raw-output
project_id = result.s
print("project id: {}".format(project_id))

# You can also specify your project id directly:
# project_id = "Your project ID"

project id: be7e848d-1a02-4282-b60a-1a728cd9cb70


## 2. Demo 1: Creating a notebook asset and running a job <a class="anchor" id="part2"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a Jupyter Notebook (.ipynb) file on your local system and you would like to run the code in the file as a job on a CPD cluster. This section shows you how to create a notebook asset and run a job on a CPD cluster. 

### 2.1 Creating a notebook asset<a class="anchor" id="part2.1"></a>

First of all, you need to create a notebook asset in your project. To create a notebook asset you need to specify:
- The environment in which your notebook is to run
- A notebook file (.ipynb).

List all the environments in your project, filter them by their display name and get the ID of the environment in which your notebook will be run:

In [12]:
environment_name = "Runtime 23.1 on Python 3.10"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [13]:
result = ! cpdctl environment list --project-id {project_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: rt231py-be7e848d-1a02-4282-b60a-1a728cd9cb70


Upload the .ipynb file:

In [14]:
remote_file_path = "notebook/cpdctl-test-notebook.ipynb"
local_file_path = "cpdctl-test-notebook.ipynb"

In [15]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {project_id}

...
OK


Create a notebook asset:

In [16]:
file_name = "cpdctl-test-notebook.ipynb"
runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)
originate = {
    'type': 'blank'
}
originate_json = json.dumps(originate)

In [17]:
result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project-id {project_id} --runtime '{runtime_json}' --originates-from '{originate_json}' --output json -j "metadata.asset_id" --raw-output
notebook_id = result.s
print("notebook id: {}".format(notebook_id))

notebook id: a8f8c4de-dad8-430e-86f7-423fbb862a42


### 2.2 Running a job <a class="anchor" id="part2.2"></a>

Before creating a notebook job, you need to create a version of your notebook:

In [18]:
result = ! cpdctl notebook version create --notebook-id {notebook_id} --output json -j "metadata.guid" --raw-output
version_id = result.s
print("version id: {}".format(version_id))

version id: 1aa9f73f-fcdd-41d0-bbbb-64105c8ee369


To create a notebook job, you need to give your job a name, add a description, and pass the notebook ID and environment ID you determined in [2.1](#part2.1). Additionally, you can add environment variables that will be used in your notebook:

In [19]:
job_name = "cpdctl-test-job"
job = {
    'asset_ref': notebook_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
            'foo=1', 
            'bar=2'
        ]
    }, 
    'description': 'my job', 
    'name': job_name
}
job_json = json.dumps(job)

In [20]:
result = ! cpdctl job create --job '{job_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: 0dd5dc64-b139-484c-be9d-b881f6554729


Run a notebook job:

In [21]:
job_run = {
    'configuration': {
        'env_variables': [
            'key1=value1', 
            'key2=value2'
        ]
    }
}
job_run_json = json.dumps(job_run)

In [22]:
result = ! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
print("run id: {}".format(run_id))

run id: a0f7b7c5-39af-4a77-acbf-b4569d54da44


You can see the output of each cell in your .ipynb file by listing job run logs:

In [23]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --project-id {project_id}

...

Cell 1:
0
1
4
9
16




## 3. Demo 2: Creating a Python script asset and running a job <a class="anchor" id="part3"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a Python script (.py) on your local system and you would like to run the code in the script as a job on a CPD cluster. This section shows you how to create a Python script asset and run a job on a CPD cluster.

### 3.1 Creating a Python script asset<a class="anchor" id="part3.1"></a>

Upload the script (.py) file:

In [24]:
remote_file_path = "script/test_script.py"
local_file_path = "test_script.py"

In [25]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {project_id}

...
OK


Specify the metadata, entity and attachments of the script file:

In [26]:
metadata = {
    "name": "my_test_script",
    "asset_type": "script",
    "asset_category": "USER",
    "origin_country": "us"
}
metadata_json = json.dumps(metadata)

entity = {
    "script": {
        "language": {
            "name": "python3"
        }
    }
}
entity_json = json.dumps(entity)

attachments = [
    {
        "asset_type": "script",
        "name": "my_test_script",
        "description": "attachment for script",
        "mime": "application/text",
        "object_key": remote_file_path
    }
]
attachments_json = json.dumps(attachments)

Create a Python script asset:

In [27]:
result = ! cpdctl asset create  --metadata '{metadata_json}' --entity '{entity_json}' --attachments '{attachments_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
script_id = result.s
print("script id: {}".format(script_id))

script id: b464cafb-63c6-4159-a5c2-14034c3c0c0d


### 3.2 Running a job<a class="anchor" id="part3.2"></a>

Similar to a notebook job, you need to specify the environment in which your script job is to run:

In [28]:
environment_name = "Runtime 23.1 on Python 3.10"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [29]:
result = ! cpdctl environment list --project-id {project_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: rt231py-be7e848d-1a02-4282-b60a-1a728cd9cb70


Now you can create a script job. To do this, you need to give your script job a name, a description, and pass the script ID and environment ID.

In [30]:
job_name = "cpdctl-test-job-for-script"
job = {
    'asset_ref': script_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
            'foo=1', 
            'bar=2'
        ]
    }, 
    'description': 'my script job', 
    'name': job_name
}
job_json = json.dumps(job)

In [31]:
result = ! cpdctl job create --job '{job_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: cf7fa3c3-b9a0-48ca-b52f-c00e8352a637


Run your script job:

In [32]:
job_run = {
    'configuration': {
        'env_variables': [
            'key1=value1', 
            'key2=value2'
        ]
    }
}
job_run_json = json.dumps(job_run)

In [33]:
result = ! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
print("run id: {}".format(run_id))

run id: 3f6050e7-decf-4dd9-bd16-043ec246849c


Show your script job run logs:

In [34]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --project-id {project_id}

...

0
1
4
9
16




## 4. Demo 3: Downloading a notebook and uploading it to another project <a class="anchor" id="part4"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a notebook in one project and would like to add a specific version of this notebook to another project. To do this, you first need to download the notebook file to your local system and then upload it to the other project. After that you need to create a notebook asset in your project by referencing the uploaded notebook file (.ipynb) and specifying the environment in which your notebook is to run.

### 4.1 Downloading a notebook <a class="anchor" id="part4.1"></a>

You can select which notebook version you want to download.

List notebook versions:

In [35]:
! cpdctl notebook version list --notebook-id {notebook_id}

...
ID                                     Created
1aa9f73f-fcdd-41d0-bbbb-64105c8ee369   1701802812862


Get the path in the storage volume to the notebook version that you want to download:

In [36]:
result = ! cpdctl notebook version list --notebook-id {notebook_id} --output json -j "(resources[].metadata.guid)[0]" --raw-output
version_id = result.s
print("version id: {}".format(version_id))

# You can also specify your version id directly:
# env_id = "Your version ID"

version id: 1aa9f73f-fcdd-41d0-bbbb-64105c8ee369


In [37]:
result = ! cpdctl notebook version get --notebook-id {notebook_id} --version-id {version_id} --output json -j "entity.file_reference" --raw-output
version_storage_path = result.s
print("version storage path: {}".format(version_storage_path))

version storage path: notebook/attachment_for_notebook_a8f8c4de_dad8_430e_86f7_423fbb862a42_1ay0zvvwelhqqbvxd9nlrc1wo.ipynb


Download the noteboook asset with the specific version from the storage path:

In [38]:
file_name = "my-new-notebook.ipynb"

! cpdctl asset file download --path {version_storage_path} --output-file {file_name} --project-id {project_id} --raw-output

...
OK
Output written to my-new-notebook.ipynb


### 4.2 Uploading the notebook to another project <a class="anchor" id="part4.2"></a>

Determine the ID of the project to which you want to upload your notebook:

In [39]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[1]" --raw-output
project2_id = result.s
print("another project id: {}".format(project2_id))

# You can also specify your another project id directly:
# project2_id = "Your another project ID"

another project id: 277c4bc5-2665-49fc-b33d-ce17fe9610fd


Upload the notebook file to this project:

In [40]:
remote_file_path = "notebook/{}".format(file_name)

! cpdctl asset file upload --path {remote_file_path} --file {file_name} --project-id {project2_id}

...
OK


After you have uploaded the notebook file to the project, you need to specify the environment in which to run the notebook:

In [41]:
environment_name = "Runtime 23.1 on Python 3.10"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [42]:
result = ! cpdctl environment list --project-id {project2_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: rt231py-277c4bc5-2665-49fc-b33d-ce17fe9610fd


Now you can create a notebook asset in this project by referencing the uploaded notebook file:

In [43]:
file_name = "my-new-notebook-in-another-project.ipynb"
runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)
originate = {
    'type': 'blank'
}
originate_json = json.dumps(originate)

In [44]:
result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project-id {project2_id} --originates-from '{originate_json}' --runtime '{runtime_json}' --output json -j "metadata.asset_id" --raw-output
notebook_id = result.s
print("notebook id: {}".format(notebook_id))

notebook id: 09348013-24cc-4ba8-8dad-e74f11d72120


## 5. Demo 4: Adding additional packages for custom environment <a class="anchor" id="part5"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a `conda-yml` file that lists your additional packages **or** you have a `pip-zip` file containing your custom packages, and you would like to install these packages in your custom environment. To do this, you need to:

- Create a custom software specification
- Add your custom packages
- Create a custom environment

### 5.1 Creating a custom software specification <a class="anchor" id="part5.1"></a>

To create a custom software specification, you need to specify the base software specification that you want to customize. You can list all the software specifications in your project and choose one of them as the base software specification:

In [45]:
! cpdctl environment software-specification list --project-id {project_id} 

...
ID                                     Name                              Created                    Description                                          Type
0062b8c9-8b7d-44a0-a9b9-46c416adcbd9   default_py3.6                     2023-10-22T10:50:09.912Z   Default Python 3.6                                   software_specification
018ebea5-1d1f-5fec-b93e-5e2ab30e7f38   runtime-22.1-r3.6                 2023-10-22T10:50:09.934Z   Software specification for IBM Runtime 22.1 on R …   software_specification
01ce9391-1a79-5a33-94fb-2e134337f314   autoai-ts_rt23.1-py3.10           2023-10-22T10:50:09.900Z   AutoAI-TimeSeries on Runtime 23.1 with Python 3.10   software_specification
020d69ce-7ac1-5e68-ac1a-31189867356a   kernel-spark3.2-scala2.12         2023-10-22T10:50:09.905Z   Jupyter kernel on Scala 2.12 with Spark 3.2          software_specification
067048c7-c771-5933-9aa0-4f9fbdc92da8   tensorflow_rt23.1-py3.10-edt      2023-10-22T10:50:09.999Z   Tensorflow 2.12 Elastic Distribute

In [46]:
base_sw_spec_name = "Runtime 23.1 on Python 3.10"
query_string = "(resources[?metadata.description == '{}'].metadata.asset_id)[0]".format(base_sw_spec_name)

result = ! cpdctl environment software-specification list --project-id {project_id} --output json -j "{query_string}" --raw-output
base_sw_spec_id = result.s
print("base software specification id: {}".format(base_sw_spec_id))

# You can also specify your base software specification id directly:
# based_sw_spec_id = "Your base software specification ID"

base software specification id: 336b29df-e0e1-5e7d-b6a5-f6ab722625b2


Create a custom software specification:

In [47]:
custom_sw_spec_name = "my_sw_spec"

base_sw_spec = {
    'guid': base_sw_spec_id
}

base_sw_spec_json = json.dumps(base_sw_spec)

sw_conf = {}
sw_conf_json = json.dumps(sw_conf)

In [48]:
result = ! cpdctl environment software-specification create --project-id {project_id} --name {custom_sw_spec_name} --base-software-specification '{base_sw_spec_json}' --software-configuration '{sw_conf_json}' --output json -j "metadata.asset_id" --raw-output
custom_sw_spec_id = result.s
print("custom software specification id: {}".format(custom_sw_spec_id))

custom software specification id: e0910504-cc78-4f93-90ac-a1f4f321923c


### 5.2 Adding additional packages <a class="anchor" id="part5.2"></a>

Create a package extension:

In [49]:
pkg_name = "my_test_packages"

In [50]:
result = ! cpdctl environment package-extension create --name {pkg_name} --type "conda_yml" --project-id {project_id}  --output json
pkg_ext_id = json.loads(result.s)['metadata']['asset_id']
print("package extension id: {}".format(pkg_ext_id))

package extension id: 58a2beda-2c3b-4c0c-9379-fe7977fc54e6


Get the path to where you want to upload the additional packages:

In [51]:
pkg_ext_href = json.loads(result.s)['entity']['package_extension']['href'].split('/')[4].split('?')[0]
remote_pkg_path = "package_extension/{}".format(pkg_ext_href)
print("path where asset should be uploaded: {}".format(remote_pkg_path))

path where asset should be uploaded: package_extension/my_test_packages_coY9oEMYz.yml


Define a conda-yaml file listing additional packages:

In [52]:
my_yaml = """
channels:
  - defaults

dependencies:
  - pip:
    - fuzzywuzzy

"""

with open('my-pkg-ext.yaml', 'w') as f:
    f.write(my_yaml)

Upload additional packages to the path returned in the previous command:

In [53]:
local_pkg_path = "./my-pkg-ext.yaml"

In [54]:
! cpdctl asset file upload --path "{remote_pkg_path}" --file {local_pkg_path} --project-id {project_id} 

...
OK


In [55]:
! cpdctl environment package-extension upload-complete --package-extension-id {pkg_ext_id} --project-id {project_id}

...
OK


Add the package extension into the custom software specification:

In [56]:
! cpdctl environment software-specification add-package-extensions --software-specification-id {custom_sw_spec_id} --package-extension-id {pkg_ext_id} --project-id {project_id}

...
OK


### 5.3 Creating a custom environment <a class="anchor" id="part5.3"></a>

List all the hardware specifications in your project and choose one that fits your custom environment:

In [57]:
! cpdctl environment hardware-specification list --project-id {project_id} 

...
ID                                     Name                     Created                    Description                                          Type
3c48c595-a6eb-5a38-bae1-7fea3f366162   CPUx2                    2023-10-22T10:50:11.641Z   2 CPU, for WML deep learning                         hardware_specification
5611c0b6-704a-5913-8766-60cbaedf5c68   GPUx1                    2023-10-22T10:50:11.642Z   1 GPU, for WML deep learning                         hardware_specification
5a1f0e64-e420-55ce-bd7a-f6d00bb942cf   ML                       2023-10-22T10:50:11.645Z   A hardware specification providing 4 CPU cores an…   hardware_specification
7b6a7c67-0727-5ff9-885e-cf3bb78ea3e2   GPUx2                    2023-10-22T10:50:11.642Z   2 GPU, for WML deep learning                         hardware_specification
8d211f8f-1ea1-5154-8e64-377521ab566e   GPUx4                    2023-10-22T10:50:11.642Z   4 GPU, for WML deep learning                         hardware_specification
93a39616-868

In [58]:
hw_spec_keyword_1 = "one CPU core"
hw_spec_keyword_2 = "4 GiB of memory"
query_string = "(resources[?contains(metadata.description, '{}') && contains(metadata.description, '{}')].metadata.asset_id)[0]".format(hw_spec_keyword_1, hw_spec_keyword_2)

result = ! cpdctl environment hardware-specification list --project-id {project_id}  --output json -j "{query_string}" --raw-output
hw_spec_id = result.s
print("hardware specification id: {}".format(hw_spec_id))

# You can also specify your hardware specification id directly:
# hw_spec_id = "Your base software specification ID"

hardware specification id: f3ebac7d-0a75-410c-8b48-a931428cc4c5


Find the ID of a runtime definition which matches the base software specification:

In [59]:
runtime_definition_description = "Runtime 23.1 on Python 3.10"
query_string = "(resources[?entity.description == '{}'].metadata.guid)[0]".format(runtime_definition_description)
result = ! cpdctl environment runtime-definition list --project-id {project_id} --output json -j "{query_string}" --raw-output
runtime_definition_id = result.s
print("runtime_definition id: {}".format(runtime_definition_id))

# You can also specify your runtime definition id directly:
# env_id = "Your runtime definition ID"

runtime_definition id: a1e0c491-2695-505d-9a9a-8a04ed215825


Create a custom environment by specifying the hardware specification, the custom software specification and the tool specification:

In [60]:
env_name = "my_custom_env"
hw_spec = {
    'guid': hw_spec_id
}
custom_sw_spec = {
    'guid': custom_sw_spec_id
}
custom_sw_spec_json = json.dumps(custom_sw_spec)
tool_spec = {
    'supported_kernels': [{
        'language': 'python', 
        'version': '3.10', 
        'display_name': 'Python 3.10'
    }]
}
hw_spec_json = json.dumps(hw_spec)
tool_spec_json = json.dumps(tool_spec)

In [61]:
result = ! cpdctl environment create --project-id {project_id} --type "notebook" --name {env_name} --display-name {env_name} --hardware-specification '{hw_spec_json}' --software-specification '{custom_sw_spec_json}' --tools-specification '{tool_spec_json}' --runtime-definition {runtime_definition_id} --output json -j "metadata.asset_id" --raw-output
custom_env_id = result.s
print("custom environment id: {}".format(custom_env_id))

custom environment id: a3848af5-602f-4edc-b56d-899faffad5d2


Now you can use this custom environment when you create a new notebook asset.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.